In [ ]:
from grid.refine import *

df = observed_rates()

In [ ]:
df.query('width > 64')

In [ ]:
df = data.load()

interp = df.interpolate().where(df.bfill().notnull())
stacked = interp.stack([1, 2, 3]).reset_index().dropna()
stacked['flops'] = flops(stacked)

best = stacked.groupby(['_time', 'boardsize']).elo.idxmax()
best = stacked.loc[best]

index = 10**np.linspace(0, np.log10(stacked.flops.max()), 1001)
regular = pd.pivot_table(stacked, 'elo', ('flops',), ('boardsize', 'width', 'depth')).ffill().reindex(index, method='nearest')
frontier = regular.expanding().max().groupby(axis=1, level=0).max().div(data.min_elos(), axis=1).mul(-1).add(1)

with plt.style.context('seaborn-poster'):
    ax = (frontier > .95).drop(11, axis=1).idxmax().plot(logy=True, marker='o', linestyle='--', grid=True)
    ax.set_ylabel('training FLOPS')
    ax.set_xlabel('boardsize')
    ax.set_title('FLOPS needed to hit 95% of perfect play is roughly linear')